Script em Python que lê os dados de uma tabela num banco MySQL, consolida os dados e salva numa outra tabela de um banco local MySQL.

In [ ]:
pip install mysql.connector

In [ ]:
pip install pandas

In [11]:
import mysql.connector as mysql
import pandas as pd

# Conexão ao banco de dados remoto
con_remote = mysql.connect(
    host="40b8f30251.nxcli.io",
    port=3306,
    user="a4f2b49a_padawan",
    password="KaratFlanksUgliedSpinal",
    database="a4f2b49a_sample_database",
)

# Conexão ao banco de dados local
con_local = mysql.connect(
    host="localhost",
    port=3306,
    user="root",
    password="tom*1005",
    database="BD_H2",
)

# Leitura dos dados da tabela "raw_data"
query = "SELECT * FROM raw_data"
df = pd.read_sql(query, con=con_remote)

# Consolidação dos dados por mês
df['mes'] = pd.to_datetime(df['datahora_acesso']).dt.month

df_consolidado = df.groupby('mes').agg(
    rake=pd.NamedAgg(column='rake', aggfunc='sum'),
    jogadores=pd.NamedAgg(column='clientes_id', aggfunc='nunique'),
    rake_cash_game=pd.NamedAgg(lambda x: x[x['modalidade'] == 'Cash Game']['rake'].sum()),
    rake_torneio=pd.NamedAgg(lambda x: x[x['modalidade'] == 'Torneio']['rake'].sum()),
    jogadores_cash_game=pd.NamedAgg(lambda x: x[x['modalidade'] == 'Cash Game']['clientes_id'].nunique()),
    jogadores_torneio=pd.NamedAgg(lambda x: x[x['modalidade'] == 'Torneio']['clientes_id'].nunique()),
)

# Criação da tabela "consolidado" no banco local
cursor = con_local.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS consolidado (
        mes INT,
        rake DECIMAL(10,2),
        jogadores INT,
        rake_cash_game DECIMAL(10,2),
        rake_torneio DECIMAL(10,2),
        jogadores_cash_game INT,
        jogadores_torneio INT
    )
""")

# Gravação dos dados consolidados no banco local
for row in df_consolidado.to_dict('records'):
    cursor.execute("""
        INSERT INTO consolidado (mes, rake, jogadores, rake_cash_game, rake_torneio, jogadores_cash_game, jogadores_torneio)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (row['mes'], row['rake'], row['jogadores'], row['rake_cash_game'], row['rake_torneio'], row['jogadores_cash_game'], row['jogadores_torneio']))

con_local.commit()

# Fechamento das conexões
con_remote.close()
con_local.close()


NotSupportedError: Authentication plugin 'caching_sha2_password' is not supported